In [2]:
import pandas as pd

# Load the JSON file into a pandas DataFrame with nested structure
df = pd.read_json('../data/Ramps New.json', orient='index')

# Set the display options to show all columns and increase the column width
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 5000


# View the first five rows of the df DataFrame with all columns
first_five_rows = df.head()
print(first_five_rows)


In [10]:
import pandas as pd
import json

# Load the JSON data into a pandas DataFrame
with open('../data/Ramps New.json') as f:
    data = json.load(f)
df = pd.json_normalize(data, 'features')

# Extract the first 5 scores and their coordinates
coordinate_score_df = df[['geometry.coordinates', 'properties.score_1', 'properties.FID']]

coordinate_score_df = coordinate_score_df.rename(columns={'geometry.coordinates': 'coordinates', 'properties.score_1': 'score_1'})
new_df = coordinate_score_df
# Print the result
print(coordinate_score_df)
point_df = coordinate_score_df[coordinate_score_df['score_1']<=10]


                                    coordinates    score_1  properties.FID
0       [-7911843.053800002, 5213855.430799998]  82.400000               0
1       [-7912104.434000002, 5213764.354100004]  92.800000               1
2      [-7912111.2201999985, 5213780.467799999]  66.341154               2
3        [-7912093.786999997, 5213788.37470001]  95.975000               3
4       [-7911853.312899999, 5213875.931500003]  87.750000               4
...                                         ...        ...             ...
26207        [-7908859.5842, 5213906.350699998]   0.000000           26207
26208        [-7908620.2472, 5213876.224299997]   0.000000           26208
26209              [-7908621.175, 5213877.0612]   0.000000           26209
26210        [-7908602.0033, 5213865.763899997]   0.000000           26210
26211        [-7910820.3533, 5209827.192599997]   0.000000           26211

[26212 rows x 3 columns]


In [17]:
import folium
import pyproj
project = pyproj.Transformer.from_crs('epsg:3857', 'epsg:4326')
point_df['latitudes'], point_df['longitudes'] = zip(*point_df['coordinates'].apply(lambda coord: project.transform(coord[0], coord[1])))

m = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

for _, row in point_df.iterrows():
    title = f"Score: {row['score_1']}"
    folium.CircleMarker(location=[row['latitudes'], row['longitudes']], popup=title,fill_color="red",color="red",fill_opacity=0.8).add_to(m)


m.save('maps/Ramp_poor_Map.html')

C:\Users\xiao\AppData\Local\Temp\ipykernel_29236\1714777107.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  point_df['latitudes'], point_df['longitudes'] = zip(*point_df['coordinates'].apply(lambda coord: project.transform(coord[0], coord[1])))


In [4]:
import folium
import pyproj
project = pyproj.Transformer.from_crs('epsg:3857', 'epsg:4326')
coordinate_score_df['latitudes'], coordinate_score_df['longitudes'] = zip(*coordinate_score_df['coordinates'].apply(lambda coord: project.transform(coord[0], coord[1])))

m = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

for _, row in coordinate_score_df.iterrows():
    title = f"Score: {row['score_1']}"
    folium.Marker(location=[row['latitudes'], row['longitudes']], popup=title).add_to(m)


m.save('maps/Ramp_Scores_Map.html')

In [5]:
import geopandas as gpd
from shapely.geometry import Point

sidewalks_gdf = gpd.read_file('../data/Sidewalks.json')
points_gdf = gpd.GeoDataFrame(
    coordinate_score_df, 
    geometry=[Point(coord) for coord in coordinate_score_df['coordinates']]
)

pd.set_option('display.max_columns', None)
joined_gdf = gpd.sjoin(points_gdf, sidewalks_gdf, op='within')
avg_scores = joined_gdf.groupby('SWK_ID', as_index=False)['score_1'].mean()
sidewalks_gdf = sidewalks_gdf.set_index('SWK_ID')
avg_scores = avg_scores.set_index('SWK_ID')
sidewalks_gdf = sidewalks_gdf.merge(avg_scores, on='SWK_ID', how='left')
sidewalks_gdf = sidewalks_gdf.reset_index()
sidewalks_gdf = sidewalks_gdf[['geometry', 'score_1', 'SWK_ID']]
print(sidewalks_gdf.columns)
sidewalks_gdf.info()
sidewalks_gdf.to_file('../data/Sidewalks_without_scores.json', driver='GeoJSON')


C:\Users\William\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\William\AppData\Local\Temp\ipykernel_16892\2661763856.py:11: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:6492

  joined_gdf = gpd.sjoin(points_gdf, sidewalks_gdf, op='within')


Index(['geometry', 'score_1', 'SWK_ID'], dtype='object')
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 23520 entries, 0 to 23519
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  23347 non-null  geometry
 1   score_1   0 non-null      float64 
 2   SWK_ID    23520 non-null  object  
dtypes: float64(1), geometry(1), object(1)
memory usage: 551.4+ KB


In [6]:
import geopandas as gpd
from shapely.geometry import Point

# Convert coordinates in new_df to same CRS as sidewalks file
new_df = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df['coordinates'].str[0], new_df['coordinates'].str[1]))
new_df.crs = {'init': 'epsg:6492'}  # replace with appropriate CRS

# Load sidewalks file as geopandas dataframe
sidewalks = gpd.read_file('../data/Sidewalks_without_scores.json')


import folium
import pandas as pd
import geopandas as gpd

# Create a map
m = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

# Add sidewalk polygons to the map
folium.Choropleth(
    geo_data=sidewalks,
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    name='Sidewalks',
    highlight=True
).add_to(m)

# Add point markers for new_df to the map
for _, row in points_gdf.iterrows():
    title = f"Score: {row['score_1']}"
    folium.Marker(location=[row['latitudes'], row['longitudes']], popup=title).add_to(m)

# Add layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('maps/test.html')


#print(sidewalks.crs)
#print(new_df.crs)
# Perform spatial join
#joined = gpd.sjoin(new_df, sidewalks, op='within')
#print(joined.columns)
#joined.info()
# Group by SWK_ID and calculate mean score_1
#result = joined.groupby('SWK_ID')['score_1'].mean()


C:\Users\William\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [7]:
sidewalks.to_file('../data/sidewalks_test.json', driver='GeoJSON')
print(points_gdf.head())

                                coordinates    score_1  properties.FID   
0   [-7911843.053800002, 5213855.430799998]  82.400000               0  \
1   [-7912104.434000002, 5213764.354100004]  92.800000               1   
2  [-7912111.2201999985, 5213780.467799999]  66.341154               2   
3    [-7912093.786999997, 5213788.37470001]  95.975000               3   
4   [-7911853.312899999, 5213875.931500003]  87.750000               4   

   latitudes  longitudes                          geometry  
0  42.352009  -71.073295  POINT (-7911843.054 5213855.431)  
1  42.351405  -71.075643  POINT (-7912104.434 5213764.354)  
2  42.351512  -71.075704  POINT (-7912111.220 5213780.468)  
3  42.351564  -71.075548  POINT (-7912093.787 5213788.375)  
4  42.352145  -71.073388  POINT (-7911853.313 5213875.932)  


In [8]:
sidewalks = gpd.read_file('../data/sidewalks_test.json')

In [9]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from scipy.spatial import KDTree
ramps = points_gdf.head(10)
ramps = ramps.rename(columns={'properties.FID':'FID'})

sidewalks['score_1'] = None

# Create a new column in the ramps dataframe to store the nearest sidewalk
ramps['nearest_sidewalk'] = None

# Loop through each ramp and calculate the distance to each sidewalk
for ramp_index, ramp_row in ramps.iterrows():
    ramp_point = ramp_row['geometry']
    min_distance = float('inf')
    nearest_sidewalk_index = None
    for sidewalk_index, sidewalk_row in sidewalks.iterrows():
        sidewalk_point = sidewalk_row['geometry']
        distance = ramp_point.distance(sidewalk_point)
        if distance < min_distance:
            min_distance = distance
            nearest_sidewalk_index = sidewalk_index
            nearest_sidewalk_id = sidewalks.at[sidewalk_index, 'SWK_ID']
    
    # Store the index of the nearest sidewalk in the ramps dataframe
    ramps.at[ramp_index, 'nearest_sidewalk'] = int(nearest_sidewalk_id)


# Join the ramps dataframe with the sidewalks dataframe to get the geometry of the nearest sidewalk
ramps = ramps.join(sidewalks.set_index(sidewalks.index), on='nearest_sidewalk', rsuffix='_nearest')

In [10]:

print(ramps)

                                coordinates    score_1  FID  latitudes   
0   [-7911843.053800002, 5213855.430799998]  82.400000    0  42.352009  \
1   [-7912104.434000002, 5213764.354100004]  92.800000    1  42.351405   
2  [-7912111.2201999985, 5213780.467799999]  66.341154    2  42.351512   
3    [-7912093.786999997, 5213788.37470001]  95.975000    3  42.351564   
4   [-7911853.312899999, 5213875.931500003]  87.750000    4  42.352145   
5            [-7911529.5534999985, 5212385]  94.700000    5  42.342247   
6   [-7911551.325599996, 5212364.080200002]  99.600000    6  42.342108   
7   [-7911523.409899997, 5212374.193499997]  86.875000    7  42.342175   
8   [-7911539.937299997, 5212356.935199998]  86.787179    8  42.342060   
9   [-7912980.275599999, 5207269.563600004]  89.106667    9  42.308272   

   longitudes                          geometry nearest_sidewalk   
0  -71.073295  POINT (-7911843.054 5213855.431)            22774  \
1  -71.075643  POINT (-7912104.434 5213764.354)  

In [54]:
ramps_gdf = gpd.read_file('../data/Ramps New.json')
sidewalks_gdf = gpd.read_file('../data/Sidewalks.json')
ramps_gdf = ramps_gdf.to_crs(sidewalks_gdf.crs)
ramps_gdf.geometry = ramps_gdf.geometry.buffer(5)

In [55]:
import json
import geojson
from shapely.geometry import shape, Point, Polygon
import geopandas as gpd


joined_gdf = gpd.sjoin(sidewalks_gdf, ramps_gdf, op='intersects')
print(joined_gdf.head())
grouped_gdf = joined_gdf.groupby('SWK_ID').agg({'FID_left': list, 'score_1': 'mean'}).reset_index()
result_gdf = sidewalks_gdf.merge(grouped_gdf, on='SWK_ID', how='left')
print(result_gdf.head())




C:\Users\William\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


      FID_left SWK_ID MATERIAL SWK_WIDTH          DISTRICT SWK_AREA  PARENT   
0            0      1       CC       5.5       CHARLESTOWN     3648  ALBIO1  \
9186      9186   9844       CC        17       CHARLESTOWN     3289  MAIN 2   
1            1    402       CC         6  SOUTH DORCHESTER     1671  ALMON1   
2            2    403       CC         6  SOUTH DORCHESTER     1351  ALMON1   
7            7    408       CC         6  SOUTH DORCHESTER     1445  ALABA2   

           SEG_ID   SIDE  CG_ID  Rpr_Yr  New_SCI  Recon_Date  HPNETWORK   
0        ALBIO1_0   LEFT  24842       0       85         NaN          0  \
9186  MAIN 2_3764   LEFT   9784       0       70         NaN          1   
1     ALMON1_1749   LEFT    401       0       55         NaN          1   
2     ALMON1_2047  RIGHT    402       0       67         NaN          1   
7        ALABA2_0  RIGHT    407    2016       86         NaN          0   

      GOODCANDID   
0              0  \
9186           0   
1             

In [77]:
new_gdf = result_gdf


In [78]:
import folium

# Create a map centered at a specific location
m = folium.Map(location=[42.361145, -71.057083], zoom_start=13)

# Add sidewalks layer to the map with color ramp based on average score
folium.Choropleth(
    geo_data=new_gdf,
    name='Average Score',
    data=new_gdf,
    columns=['SWK_ID', 'score_1'],
    key_on='properties.SWK_ID',
    fill_color='RdYlBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Score',
    reverse=True,
    highlight=True,
    show=False
).add_to(m)

print(new_gdf['score_1'].isna().sum())
new_gdf = new_gdf.dropna(subset=['geometry'])

# Adds the ability to click the sidewalk to see the score for that sidewalk
folium.GeoJson(
    new_gdf,
    name='Sidewalk Scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['SWK_ID'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=['score_1'], labels=True, sticky=True),
    show=False
).add_to(m)




# # Add a layer control to the map
# folium.LayerControl().add_to(m)

# # Save the map as an HTML file
# m.save('sidewalks_map.html')

4256


In [79]:
# boston_map = folium.Map(location=[42.361145, -71.057083], zoom_start=13)

# Add sidewalks layer to the map with color ramp based on average score
# folium.Choropleth(
#     geo_data=sidewalks_gdf,
#     name='Average Score',
#     data=sidewalks_gdf,
#     columns=['SWK_ID', 'score_1'],
#     key_on='feature.id',
#     fill_color='YlGn',
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name='Average Score'
# ).add_to(boston_map)

# # Add a layer control to the map
# folium.LayerControl().add_to(boston_map)

# # Save the map as an HTML file
# boston_map.save('sidewalks_map.html')

In [80]:
# # Load the GeoJSON data
# with open('Boston_Neighborhoods.geojson') as f:
#     data = json.load(f)

# # Create a Folium map centered on Boston
# m = folium.Map(location=[42.3601, -71.0589], zoom_start=11)

# # Add the GeoJSON data as a Choropleth layer
# folium.Choropleth(
#     geo_data=data,
#     name='choropleth',
#     data=None,
#     columns=None,
#     key_on='feature.properties.Name',
#     fill_color='YlGn',
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name='Boston Neighborhoods',
#     smooth_factor=0
# ).add_to(m)

# # Save the map as an HTML file
# m.save('boston_neighborhoods_map.html')

In [81]:
neighborhoods_gdf = gpd.read_file('../data/Boston_Neighborhoods.geojson')
sidewalk_gdf = new_gdf.to_crs(neighborhoods_gdf.crs)


In [82]:
joined_gdf = gpd.sjoin(sidewalk_gdf, neighborhoods_gdf, how='left', op='within')

neighborhood_scores = joined_gdf.groupby('Name')['score_1'].mean().reset_index()
print(neighborhood_scores)
neighborhoods_with_scores_gdf = neighborhoods_gdf.merge(neighborhood_scores, on='Name', how='left')




C:\Users\William\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


                       Name    score_1
0                   Allston  82.655438
1                  Back Bay  68.652221
2               Bay Village  60.169413
3               Beacon Hill  46.428411
4                  Brighton  81.199491
5               Charlestown  74.763348
6                 Chinatown  71.930417
7                Dorchester  79.388282
8                  Downtown  71.080202
9               East Boston  73.536719
10                   Fenway  80.040229
11                Hyde Park  72.843757
12            Jamaica Plain  75.542225
13         Leather District  82.442910
14                 Longwood  82.741270
15                 Mattapan  77.227399
16             Mission Hill  70.300619
17                North End  67.804723
18               Roslindale  74.020498
19                  Roxbury  75.934348
20             South Boston  80.009617
21  South Boston Waterfront  81.672381
22                South End  72.835833
23                 West End  81.083436
24             West Roxbu

In [83]:

# Add sidewalks layer to the map with color ramp based on average score
folium.Choropleth(
    geo_data=neighborhoods_with_scores_gdf,
    name='Neighborhood Colors',
    data=neighborhoods_with_scores_gdf,
    columns=['Name', 'score_1'],
    key_on='feature.properties.Name',
    fill_color='RdYlBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Score',
    bins=list(range(0, 101, 10)),
    reverse=True,
    highlight=True,
    show=False
).add_to(m)

print(neighborhoods_with_scores_gdf['score_1'].isna().sum())


# Adds the ability to click the sidewalk to see the score for that sidewalk
folium.GeoJson(
    neighborhoods_with_scores_gdf,
    name='Neighborhood Scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['Name'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=['score_1'], labels=True, sticky=True),
    show=False
).add_to(m)




# Add a layer control to the map
folium.LayerControl().add_to(m)

#add color fillter for color ramps on map
folium.plugins.DualMap([42.361145, -71.057083], zoom_start=13, tiles='Stamen Terrain').add_to(m)
#choose red ramp for color
folium.plugins.ColorRamp(color_ramp='Reds').add_to(m)

# Save the map as an HTML file
m.save('maps/neighborhood_scores.html')

1
